# [Practice] Sentiment Analysis Practice 
- 한국 식당 리뷰 감성분석
- <커널 재시작시 여기서부터 시작> 부분으로 이동해서 실행

---

### 데이터 로드

데이터 출처: https://huggingface.co/datasets/leey4n/KR3

In [1]:
from datasets import load_dataset

kr3 = load_dataset("leey4n/KR3", split='train')
kr3 = kr3.remove_columns(['__index_level_0__']) # Original file didn't include this column. Suspect it's a hugging face issue.

# 레이블이 2(모호한 답변)인 리뷰 drop
kr3_binary = kr3.filter(lambda example: example['Rating'] != 2)

In [2]:
import pandas as pd

# 'reviews' 코퍼스와 'ratings' 라벨 저장
reviews = kr3_binary["Review"]
ratings = kr3_binary['Rating']

# 샘플 확인
review_df = pd.DataFrame(kr3_binary, columns=kr3_binary.column_names)
display(review_df)

,Rating,Review
0,1,숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편...
1,1,고기가 정말 맛있었어요! 육즙이 가득 있어서 너무 좋았아요 일하시는 분들 너무 친절...
2,1,"잡내 없고 깔끔, 담백한 맛의 순댓국이 순댓국을 안 좋아하는 사람들에게도 술술 넘어..."
3,1,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다 맛도 좋습니다 다만 양념장이 ...
4,1,순댓국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요. 그러나 밥 양...
...,...,...
459016,0,"731 배달 시켜 먹었고요, 거리상 1.8km입니다. 배민에서 시켰고 정확히 58만..."
459017,1,송탄 미군부대 근처에 위치한 곳 원래 로컬 맛 집으로 되게 유명했는데 삼대 천왕에 ...
459018,1,집에서 40킬로 정도 떨어져 있는 곳인데도 몇 달에 한 번은 이거 먹으러 일부러 갑...
459019,0,원래 글 안 쓰는데 이거는 정말 다른 분들 위해서 써야 할 것 같네요 방금 포장 주...


In [3]:
review_df['Review'][2]

'잡내 없고 깔끔, 담백한 맛의 순댓국이 순댓국을 안 좋아하는 사람들에게도 술술 넘어갈 듯합니다. 정식 메뉴를 시키면 구성비 좋게 찹쌀순대와 수육이 나오는데 아주 푸짐해요. 해장하러 갔는데 국물이 고소해서 소주를 더 시키게 되는... 여하튼 순댓국이 일품입니다. 송파에선 알아주는 터줏대감인듯해요'

In [4]:
# 긍/부정 비율 확인 (긍정이 압도적으로 많음)
positive_rate = len(review_df[review_df['Rating'] == 1]) / len(review_df)
negative_rate = 1 - positive_rate
print("긍정 리뷰 비율:", positive_rate, "|", "부정 리뷰 비율:", negative_rate)

긍정 리뷰 비율: 0.8455190503266735 | 부정 리뷰 비율: 0.15448094967332648


---

### 데이터 전처리

In [5]:
import re

# 1) 안전 불용어(기능어 중심)
safe_stopwords_ko = set("""
은 는 이 가 을 를 에 의 와 과 도 만 보다 에서 에게 께 부터 까지 마다 한테 처럼 대로 하고 으로 로 으로써 로써
그리고 그러나 하지만 또는 그래서 그런데 혹은 따라서 그러므로 고로
저 나 너 우리 저희 너희 그 이 저기 거기 여기 무엇 무슨 누구 어디 어느 어떤 이것 그것 저것 이런 그런 저런 이번 이쪽 저쪽 그쪽 이때 그때
다 요 죠 네 데 게 지 군 나 니다 합니다 했다 한다 된다 였다
때 시간 동안 시각 무렵 이상 이하 전후 이래 이후
또 또한 그냥 다시 계속 항상 여전히 아직 가끔 자주 거의
아 어 야 응 음 흠 헐 우와 와 오 이야 휴 허 하하 ㅋㅋ ㅎㅎ ㅠㅠ
""".split())

# 2) 보호 단어(감성 핵심)
protect_sentiment_words = set("""
별로 좋아 좋다 최고 최악 실망 만족 불만 추천 재방문
맛있다 맛없다 싱겁다 짜다 달다 매콤 맵다 느끼하다 담백하다 신선 싱싱 비린 눅눅 바삭 촉촉 부드럽다 질기다
양 많다 적다 푸짐 부족 비싸다 싸다 가성비 가심비
빠르다 느리다 한산 복잡 붐빈다 대기 기다리다
깨끗 청결 더럽 위생 지저분 냄새 쾌적 시끄럽다 조용하다
서비스 응대 친절 불친절 상냥 무례 거칠다
정말 진짜 너무 매우 아주 꽤 완전
안 못 없다 없음 아니라 비추 다시는 다신
""".split())

# 3) 1글자 화이트리스트(도메인 핵심)
whitelist_1char = set(list("맛밥국탕면롤육쌈김술밥빵밥밥양밥밥")) | {"맛","밥","국","양","술","김"}

neg_prefix = {"안","못","안함","않","못함","무","미","불","무척"}  # 필요시 조정
emphasis_words = {"정말","진짜","너무","매우","아주","완전","꽤"}

def normalize_token(t: str) -> str:
    # 숫자 통일 (선택)
    if re.fullmatch(r"\d+", t):
        return "NUM"
    # ㅋㅋㅋ, ㅎㅎㅎ 축약 (선택)
    t = re.sub(r"(ㅋ)\1+", r"\1\1", t)
    t = re.sub(r"(ㅎ)\1+", r"\1\1", t)
    return t

def join_negation(tokens):
    """'안/못/없다' + (다음 감성 단어) -> 결합 토큰 생성"""
    joined = []
    i = 0
    while i < len(tokens):
        t = tokens[i]
        if t in {"안","못","없다","없음","아니다","별로"} and i+1 < len(tokens):
            nxt = tokens[i+1]
            # 결합 토큰 만들기
            joined.append(f"{t}_{nxt}")
            i += 2
            continue
        joined.append(t)
        i += 1
    return joined

def filter_tokens(tokens,
                  stopwords=safe_stopwords_ko,
                  protect=protect_sentiment_words,
                  min_len=1):
    out = []
    tokens = [normalize_token(t) for t in tokens]
    tokens = join_negation(tokens)

    for t in tokens:
        if t in protect or t in emphasis_words:
            out.append(t)                         # 보호
            continue
        if len(t) < min_len and t not in whitelist_1char:
            continue                              # 1글자 필터 (화이트리스트 제외)
        if t in stopwords:
            continue                              # 불용어 제거
        out.append(t)
    return out

In [6]:
from konlpy.tag import Okt
from nltk import word_tokenize, sent_tokenize
from tqdm import tqdm

okt = Okt()

preprocessed_sentences = []

for review in tqdm(reviews):
    review = re.sub(r"[^가-힣0-9\s]", "", review)   # 구두점, 특수문자, 영문 제거
    review = re.sub(r"\s+", " ", review).strip()   # 여러 공백을 하나의 공백으로
    tokens = okt.morphs(review, stem=True, norm=True) # 토큰화
    tokens = filter_tokens(tokens)
    preprocessed_sentences.append(tokens)

100%|██████████| 459021/459021 [2:22:51<00:00, 53.55it/s]   


In [7]:
preprocessed_sentences[:2]    

[['숙성',
  '돼지고기',
  '전문점',
  '이다',
  '건물',
  '모양',
  '때문',
  '매장',
  '모양',
  '좀',
  '특이하다',
  '쾌적하다',
  '편이',
  '고',
  '살짝',
  '레트로',
  '감성',
  '분위기',
  '잡다',
  '모든',
  '직원',
  '분들',
  '께서',
  '전부',
  '가능하다',
  '멘트',
  '치다',
  '고기',
  '초반',
  '커팅',
  '까지는',
  '굽다',
  '가격',
  '저렴하다',
  '편',
  '아니다_맛',
  '준수',
  '하다',
  '등심',
  '덧',
  '살이',
  '인상',
  '깊다',
  '구이',
  '별로_일',
  '줄',
  '알다',
  '육',
  '향',
  '짙다',
  '얇다',
  '며',
  '뻑뻑',
  '하다',
  '않다',
  '하이라이트',
  '된장찌개',
  '진짜',
  '굿',
  '이다',
  '버터',
  '간장',
  '밥',
  '골뱅이',
  '국수',
  '등',
  '나중',
  '더',
  '맛보다',
  '하다',
  '것',
  '들',
  '남기다',
  '두다'],
 ['고기',
  '정말',
  '맛있다',
  '육즙',
  '가득',
  '있다',
  '너무',
  '좋다',
  '일',
  '하다',
  '분들',
  '너무',
  '친절하다',
  '좋다',
  '가격',
  '조금',
  '있다',
  '그만하다',
  '맛',
  '이라고',
  '생각']]

preprocessed_sentences 저장 & 로드 
- 커널 재시작시 시간 줄이기용

In [8]:
import json, gzip

# 저장 (리뷰마다 한 줄)
with gzip.open("preprocessed_sentences.jsonl.gz", "wt", encoding="utf-8") as f:
    for toks in preprocessed_sentences:
        json.dump({"tokens": toks}, f, ensure_ascii=False)
        f.write("\n")

---

# 커널 재시작시 여기서부터 시작

토큰화 및 전처리 완료된 corpus 로드

In [9]:
# preprocessed_sentences 로드
preprocessed_sentences = []
with gzip.open("preprocessed_sentences.jsonl.gz", "rt", encoding="utf-8") as f:
    for line in f:
        preprocessed_sentences.append(json.loads(line)["tokens"])

print(preprocessed_sentences[:2])

[['숙성', '돼지고기', '전문점', '이다', '건물', '모양', '때문', '매장', '모양', '좀', '특이하다', '쾌적하다', '편이', '고', '살짝', '레트로', '감성', '분위기', '잡다', '모든', '직원', '분들', '께서', '전부', '가능하다', '멘트', '치다', '고기', '초반', '커팅', '까지는', '굽다', '가격', '저렴하다', '편', '아니다_맛', '준수', '하다', '등심', '덧', '살이', '인상', '깊다', '구이', '별로_일', '줄', '알다', '육', '향', '짙다', '얇다', '며', '뻑뻑', '하다', '않다', '하이라이트', '된장찌개', '진짜', '굿', '이다', '버터', '간장', '밥', '골뱅이', '국수', '등', '나중', '더', '맛보다', '하다', '것', '들', '남기다', '두다'], ['고기', '정말', '맛있다', '육즙', '가득', '있다', '너무', '좋다', '일', '하다', '분들', '너무', '친절하다', '좋다', '가격', '조금', '있다', '그만하다', '맛', '이라고', '생각']]


단어사전 만들기

In [10]:
from collections import Counter
import json

SPECIALS = ["<pad>", "<unk>"]  # 0: pad, 1: unk

class SimpleVocab:
    def __init__(self, itos, unk_token="<unk>"):
        self._itos = list(itos)                       # index -> token
        self._stoi = {t:i for i,t in enumerate(itos)} # token -> index
        self.unk = unk_token
        self.unk_id = self._stoi.get(unk_token, 1)

    def __len__(self): return len(self._itos)
    def lookup_indices(self, tokens): return [self._stoi.get(t, self.unk_id) for t in tokens]
    def lookup_token(self, idx): return self._itos[idx]
    def get_stoi(self): return self._stoi
    def get_itos(self): return self._itos

    def save(self, path):
        with open(path, "w", encoding="utf-8") as f:
            json.dump({"itos": self._itos, "unk": self.unk}, f, ensure_ascii=False)

    @staticmethod
    def load(path):
        with open(path, "r", encoding="utf-8") as f:
            obj = json.load(f)
        return SimpleVocab(obj["itos"], unk_token=obj.get("unk","<unk>"))

def build_vocab_simple(tokenized_sentences, min_freq=2, max_size=None, specials=SPECIALS):
    """
    min_freq: 최소 등장 빈도 (희귀어를 <unk>로 처리)
    max_size: 특수토큰 포함한 전체 vocab 최대 크기 (None이면 제한 없음)
    정렬: 빈도 descending, 동률이면 사전순
    """
    counter = Counter()
    for sent in tokenized_sentences:
        counter.update(sent)

    # 빈도 필터
    items = [(w, c) for w, c in counter.items() if c >= min_freq]
    # 빈도 내림차순, 동률 사전순
    items.sort(key=lambda x: (-x[1], x[0]))

    # 특수토큰 먼저, 그 뒤 단어들
    words = [w for w, _ in items]
    itos = list(specials) + words

    # max_size 제한 (특수토큰 포함하여 자르기)
    if max_size is not None and len(itos) > max_size:
        itos = itos[:max_size]

    return SimpleVocab(itos, unk_token="<unk>")

In [11]:
vocab = build_vocab_simple(preprocessed_sentences, min_freq=2, max_size=30000)
ids = vocab.lookup_indices(preprocessed_sentences[0])
print(len(vocab), ids[:20], [vocab.lookup_token(i) for i in ids[:10]])

30000 [929, 604, 701, 9, 592, 1177, 160, 244, 1177, 38, 492, 1306, 217, 41, 190, 3565, 1172, 43, 388, 364] ['숙성', '돼지고기', '전문점', '이다', '건물', '모양', '때문', '매장', '모양', '좀']


vocab 만든거 저장

In [12]:
vocab.save("vocab.json")

vocab 다시 로드해서 사용

In [13]:
vocabulary = SimpleVocab.load("vocab.json")

In [14]:
# 시퀀싱
sequenced_tokens = [vocab.lookup_indices(tokens) for tokens in preprocessed_sentences]

패딩

In [15]:
import numpy as np

lengths = [len(seq) for seq in sequenced_tokens]

print("최소 길이:", np.min(lengths))
print("최대 길이:", np.max(lengths))
print("평균 길이:", np.mean(lengths))
print("중앙값:", np.median(lengths))
print("95% 분위수:", np.percentile(lengths, 95))
print("99% 분위수:", np.percentile(lengths, 99))

최소 길이: 0
최대 길이: 1169
평균 길이: 38.26029745915764
중앙값: 21.0
95% 분위수: 130.0
99% 분위수: 256.0


In [16]:
ratings = list(kr3_binary["Rating"])   # 정답 라벨 (0/1)

# 입력-라벨 동기화 (빈 시퀀스 제거)
filtered_pairs = [(seq, y) for seq, y in zip(sequenced_tokens, ratings) if len(seq) > 0]

sequenced_tokens, labels = map(list, zip(*filtered_pairs))

In [17]:
print(len(sequenced_tokens), len(labels))
print(sequenced_tokens[0])  
print(labels[0])         

455822 455822
[929, 604, 701, 9, 592, 1177, 160, 244, 1177, 38, 492, 1306, 217, 41, 190, 3565, 1172, 43, 388, 364, 82, 161, 688, 828, 288, 4788, 697, 39, 2623, 4838, 1580, 183, 26, 231, 211, 2413, 3284, 2, 641, 8094, 711, 422, 476, 298, 4714, 116, 150, 707, 93, 4068, 495, 1055, 2750, 2, 13, 4736, 801, 37, 484, 9, 340, 466, 54, 3508, 367, 173, 783, 29, 269, 2, 10, 14, 637, 306]
1


In [18]:
def pad(ids, max_len, pad_id=0):
    return ids[:max_len] + [pad_id] * max(0, max_len - len(ids))

max_len = 130  # 95% 분위
pad_id = vocab.get_stoi()["<pad>"]

In [19]:
padded_sequences = [pad(seq, max_len, pad_id) for seq in sequenced_tokens]

In [20]:
print(padded_sequences[0])
print(len(padded_sequences[0]))     # max_len = 130 확인
print("pad id:", pad_id)        

[929, 604, 701, 9, 592, 1177, 160, 244, 1177, 38, 492, 1306, 217, 41, 190, 3565, 1172, 43, 388, 364, 82, 161, 688, 828, 288, 4788, 697, 39, 2623, 4838, 1580, 183, 26, 231, 211, 2413, 3284, 2, 641, 8094, 711, 422, 476, 298, 4714, 116, 150, 707, 93, 4068, 495, 1055, 2750, 2, 13, 4736, 801, 37, 484, 9, 340, 466, 54, 3508, 367, 173, 783, 29, 269, 2, 10, 14, 637, 306, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
130
pad id: 0


Torch Tensor 변환

In [21]:
import torch

inputs = torch.tensor(padded_sequences, dtype=torch.long)           # shape: (N, T=130)
labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)     # shape: [N, 1]

---

### 모델 생성 및 학습

모델 정의

모델 인스턴스 생성

모델 학습